In [2]:
import pandas as pd
from transformers import pipeline
import nltk

c:\Users\User\.conda\envs\UW\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\User\.conda\envs\UW\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\User\.conda\envs\UW\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\User\.conda\envs\UW\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BART_classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')
#mDeBERTa_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

In [4]:
#df=pd.read_csv('india-news-headlines.csv')
df2 = pd.read_csv('datasets/US_stocks.csv')

df2.drop(['Ticker', 'Company Name', 'Comment'], inplace=True, axis=1)

df2_sector = df2.drop(['Industry Group', 'Industry', 'Sub-Industry'], axis=1)
df2_industry_group = df2.drop(['Sector', 'Industry', 'Sub-Industry'], axis=1)
df2_industry = df2.drop(['Industry Group', 'Sector', 'Sub-Industry'], axis=1)
df2_sub_industry = df2.drop(['Industry Group', 'Industry', 'Sector'], axis=1)

#df2.head(n=10)
#df2_sector.head(n=10)
#df2_industry_group.head(n=10)
#df2_industry.head(n=10)
#df2_sub_industry.head(n=10)
#df2_sector.values[0]

num_sector=len(df2_sector['Sector'].unique())
num_industry_group=len(df2_industry_group['Industry Group'].unique())
num_industry=len(df2_industry['Industry'].unique())
num_sub_industry=len(df2_sub_industry['Sub-Industry'].unique())

print(f'Sectors:{num_sector}\nIndustry Groups:{num_industry_group}\nIndustries:{num_industry}\nSub-Industries:{num_sub_industry}')
len(df2_sector['Description'].iloc[0])



Sectors:11
Industry Groups:24
Industries:68
Sub-Industries:155


995

In [22]:
#Predict Sector with 1st version BART

labels = df2_sector['Sector'].unique()
text = df2_sector['Description'].head(n=3)

hypothesis_template = 'This text is about {}.'
predicted_labels=[]
for sentence in text:

    prediction = BART_classifier(sentence, labels, hypothesis_template=hypothesis_template, multi_label=True)
    
    predicted_labels.append(prediction['labels'][0])

In [5]:
#Text tests
from nlp_utils import *
import math

#df2_sector['Description'].iloc[0]
#df2_sector.iloc[0]
dt_text_things = add_text_length(df2_sector, 'Description')
#max_len = math.floor(dt_text_things['word_count'].mean())
max_len = dt_text_things['word_count'].m
lst=bart(df2_sector['Description'].iloc[0:3], max_len)

#a=nltk.word_tokenize(df2_sector['Description'].iloc[4])
#print(len(a))



                            min        mean      max
char_count           145.000000  829.929707  886.000
word_count            29.000000  156.594148  212.000
sentence_count         2.000000    6.275763   12.000
avg_word_length        3.985849    5.319662    6.928
avg_sentence_lenght   13.625000   26.273055   90.000


NameError: name 'max_len' is not defined

In [59]:
#Text cleanning

lst

[' Corteva, Inc. is a global provider of seed and crop protection solutions focused on the agriculture industry. Its seed segment is engaged in developing and supplying germplasm and traits that produce optimum yields for farms around the world. Its crop protection segment provides herbicides, insecticides, nitrogen stabilizers and pasture and range management herbicides.',
 " Alico, Inc. is an agribusiness and land management company. The Company's principal lines of business are citrus groves and conservation. The Alico Citrus segment owns and manages citrus land in DeSoto, Polk, Collier, Hendry, Charlotte, Highlands, and Hardee Counties in Florida. The Land Management and Other Operations segment is engaged in land leasing for recreational and grazing purposes.",
 ' Limoneira has three business divisions, such as agribusiness, rental operations, and real estate development. It markets and sells citrus directly to food service, wholesale, and retail customers. The rental operations d

In [ ]:
#Predict Sector with 2nd version BART

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("AyoubChLin/Bart-MNLI-CNN_news")

model = AutoModelForSequenceClassification.from_pretrained("AyoubChLin/Bart-MNLI-CNN_news")
classifier = pipeline(
    "zero-shot-classification",
    model=model,
    tokenizer=tokenizer,
    device=0
)

In [28]:
print(predicted_labels)
print(df2_sector.head(n=3))

['Industrials', 'Industrials', 'Industrials']
                                         Description            Sector
0  Corteva, Inc. is a global provider of seed and...         Materials
1  Alico, Inc. is an agribusiness and land manage...  Consumer Staples
2  Limoneira Company is primarily an agribusiness...  Consumer Staples
